In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os

In [2]:
data = pd.read_csv("anomaly_data.csv")

In [3]:
data.head()

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,192.168.0.13-192.168.0.16-10000-10101-17,192.168.0.13,10000,192.168.0.16,10101,17,25/07/2019 03:25:53 AM,75,1,1,...,0.0,0,0,75.0,0.000000,75,75,Anomaly,Mirai,Mirai-Ackflooding
1,192.168.0.13-222.160.179.132-554-2179-6,222.160.179.132,2179,192.168.0.13,554,6,26/05/2019 10:11:06 PM,5310,1,2,...,0.0,0,0,2655.0,2261.327486,4254,1056,Anomaly,DoS,DoS-Synflooding
2,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,11/7/2019 1:24,141,0,3,...,0.0,0,0,70.5,0.707107,71,70,Anomaly,Scan,Scan Port OS
3,192.168.0.13-192.168.0.16-9020-52964-6,192.168.0.16,52964,192.168.0.13,9020,6,4/9/2019 3:58,151,0,2,...,0.0,0,0,151.0,0.000000,151,151,Anomaly,Mirai,Mirai-Hostbruteforceg
4,192.168.0.1-239.255.255.250-36763-1900-17,192.168.0.1,36763,239.255.255.250,1900,17,10/9/2019 1:41,153,2,1,...,0.0,0,0,76.5,0.707107,77,76,Anomaly,Mirai,Mirai-Hostbruteforceg


In [4]:
data.shape

(625783, 86)

In [5]:
null_check = data.isnull().sum()

null_check.to_csv('null_count')
#no null data has been found

In [6]:
data['Label'].unique()

array(['Anomaly', 'Normal'], dtype=object)

In [7]:
data['Fwd_PSH_Flags'].unique()

array([0], dtype=int64)

In [8]:
data['Bwd_PSH_Flags'].unique()

array([0, 1], dtype=int64)

In [9]:
data['Fwd_URG_Flags'].unique()

array([0], dtype=int64)

In [10]:
data['Bwd_URG_Flags'].unique()

array([0, 1], dtype=int64)

In [11]:
data['FIN_Flag_Cnt'].unique()

array([0, 1], dtype=int64)

In [12]:
data['RST_Flag_Cnt'].unique()

array([0, 1], dtype=int64)

In [13]:
data['URG_Flag_Cnt'].unique()

array([0, 1], dtype=int64)

In [14]:
data['CWE_Flag_Count'].unique()

array([0, 1], dtype=int64)

In [15]:
data['ECE_Flag_Cnt'].unique()

array([0, 1], dtype=int64)

In [16]:
data['Down/Up_Ratio'].unique()

array([ 1,  2,  0,  3,  4,  5, 10,  7,  8,  6, 13, 12, 11,  9, 14],
      dtype=int64)

In [17]:
data['Fwd_Byts/b_Avg'].unique()

array([0], dtype=int64)

In [18]:
data['Fwd_Pkts/b_Avg'].unique()

array([0], dtype=int64)

In [19]:
data['Fwd_Blk_Rate_Avg'].unique()

array([0], dtype=int64)

In [20]:
data['Bwd_Byts/b_Avg'].unique()

array([0], dtype=int64)

In [21]:
data['Bwd_Pkts/b_Avg'].unique()

array([0], dtype=int64)

In [22]:
data['Bwd_Blk_Rate_Avg'].unique()

array([0], dtype=int64)

In [23]:
data['Init_Fwd_Win_Byts'].unique()

array([-1], dtype=int64)

In [24]:
data['Fwd_Seg_Size_Min'].unique()

array([0], dtype=int64)

In [25]:
print(data[data.Label=='Anomaly'].shape)
print(data[data.Label=='Normal'].shape)

(585710, 86)
(40073, 86)


In [26]:
temp_ignore_cols = set(['Flow_ID', 'Label', 'Fwd_PSH_Flags', 'Fwd_URG_Flags', 'Fwd_Byts/b_Avg', 
                        'Fwd_Pkts/b_Avg', 'Fwd_Blk_Rate_Avg','Bwd_Byts/b_Avg', 'Bwd_Pkts/b_Avg', 'Bwd_Blk_Rate_Avg',
                        'Init_Fwd_Win_Byts', 'Fwd_Seg_Size_Min', 'Cat', 'Sub_Cat'])

In [27]:
cols = set(data.columns)

In [28]:
TRAIN_COLS = list(cols-temp_ignore_cols)
TG_COL = ["Label"]
print("Number of Features :", len(TRAIN_COLS))

Number of Features : 72


In [29]:
corr = data[TRAIN_COLS+TG_COL].corr()
corr

,Subflow_Fwd_Byts,Idle_Mean,Flow_Duration,Pkt_Len_Std,Src_Port,Tot_Fwd_Pkts,Flow_Byts/s,Subflow_Fwd_Pkts,Active_Std,Bwd_Header_Len,...,Fwd_IAT_Tot,Flow_Pkts/s,Bwd_IAT_Mean,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Mean,Bwd_Pkts/s,Protocol,Bwd_Pkt_Len_Std,Fwd_IAT_Max,Flow_IAT_Std
Subflow_Fwd_Byts,1.000000,-0.053663,0.089187,0.047819,-0.257147,0.191263,0.195484,0.191263,0.174905,-0.153132,...,0.203613,-0.078895,-0.076257,0.094838,0.719441,-0.122668,0.120057,-0.129610,0.106869,0.031981
Idle_Mean,-0.053663,1.000000,0.802833,-0.049587,-0.220348,-0.062657,-0.128176,-0.062657,0.073413,0.064365,...,0.336481,-0.080629,0.833780,-0.079914,-0.096172,-0.077496,-0.136791,-0.030122,0.358945,0.546918
Flow_Duration,0.089187,0.802833,1.000000,0.002597,-0.151521,0.001276,-0.096971,0.001276,0.337718,0.132403,...,0.715144,-0.060482,0.746066,0.020273,-0.054266,-0.059062,-0.086556,0.007581,0.690755,0.743892
Pkt_Len_Std,0.047819,-0.049587,0.002597,1.000000,0.023446,-0.075927,0.035190,-0.075927,0.023750,0.128655,...,0.045097,-0.116850,-0.041346,0.127303,-0.024389,-0.082737,-0.125368,0.716503,0.043067,0.032920
Src_Port,-0.257147,-0.220348,-0.151521,0.023446,1.000000,0.204387,0.012914,0.204387,-0.011144,-0.099951,...,-0.017139,0.296473,-0.200150,-0.002298,-0.428869,0.239106,0.463515,0.165690,-0.015297,-0.020744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bwd_Pkts/s,-0.122668,-0.077496,-0.059062,-0.082737,0.239106,0.011028,0.231854,0.011028,-0.005655,-0.119165,...,-0.015271,0.797309,-0.066737,-0.094187,-0.154685,1.000000,0.280720,-0.035470,-0.014689,-0.018549
Protocol,0.120057,-0.136791,-0.086556,-0.125368,0.463515,0.305596,-0.009692,0.305596,0.001321,-0.448174,...,0.003143,0.371634,-0.139607,-0.206127,0.032943,0.280720,1.000000,-0.200733,0.002893,-0.012358
Bwd_Pkt_Len_Std,-0.129610,-0.030122,0.007581,0.716503,0.165690,-0.104060,0.014225,-0.104060,0.019822,0.236067,...,0.030682,-0.075876,-0.017091,0.119872,-0.167763,-0.035470,-0.200733,1.000000,0.031536,0.025126
Fwd_IAT_Max,0.106869,0.358945,0.690755,0.043067,-0.015297,0.039186,-0.023357,0.039186,0.291235,0.076976,...,0.967051,-0.014181,0.270928,0.047344,0.011474,-0.014689,0.002893,0.031536,1.000000,0.649577


In [30]:
corr.to_csv('correlation_matrix')

In [31]:
print(cols)

{'Subflow_Fwd_Byts', 'Idle_Mean', 'Tot_Fwd_Pkts', 'Subflow_Fwd_Pkts', 'Bwd_Header_Len', 'Idle_Std', 'Down/Up_Ratio', 'Flow_IAT_Max', 'Fwd_IAT_Std', 'SYN_Flag_Cnt', 'PSH_Flag_Cnt', 'Flow_ID', 'Fwd_Pkt_Len_Max', 'Dst_Port', 'Bwd_IAT_Min', 'Init_Bwd_Win_Byts', 'Flow_IAT_Mean', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Std', 'Pkt_Len_Max', 'URG_Flag_Cnt', 'TotLen_Fwd_Pkts', 'Flow_IAT_Min', 'Pkt_Len_Var', 'ACK_Flag_Cnt', 'Bwd_Pkts/b_Avg', 'Fwd_Pkt_Len_Min', 'Subflow_Bwd_Byts', 'Sub_Cat', 'Pkt_Len_Mean', 'Bwd_Pkt_Len_Min', 'Fwd_Act_Data_Pkts', 'Fwd_Byts/b_Avg', 'Fwd_Seg_Size_Min', 'Idle_Min', 'Bwd_IAT_Std', 'Fwd_Pkts/s', 'Fwd_IAT_Mean', 'Active_Max', 'Fwd_Header_Len', 'Fwd_IAT_Tot', 'Flow_Pkts/s', 'Bwd_IAT_Mean', 'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Mean', 'Bwd_Pkts/s', 'Timestamp', 'Bwd_Pkt_Len_Std', 'Fwd_IAT_Max', 'Flow_IAT_Std', 'Bwd_Blk_Rate_Avg', 'Src_IP', 'Flow_Duration', 'Pkt_Len_Std', 'Src_Port', 'Flow_Byts/s', 'Active_Std', 'Fwd_PSH_Flags', 'Init_Fwd_Win_Byts', 'Bwd_URG_Flags', 'Active_Mean', 'Fwd

In [32]:
# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

In [33]:
data['Src_IP']= label_encoder.fit_transform(data['Src_IP'])

In [34]:
data['Dst_IP']= label_encoder.fit_transform(data['Dst_IP'])

In [35]:
data['Timestamp']= label_encoder.fit_transform(data['Timestamp'])

In [36]:
data['Label']= label_encoder.fit_transform(data['Label'])

In [37]:
data.head()
data.shape


(625783, 86)

In [38]:
train_data = data[TRAIN_COLS+TG_COL]


In [39]:
train_data.head()
train_data.shape

(625783, 73)

In [40]:
import numpy as np

train_data= train_data[np.isfinite(train_data).all(1)]
# train_label = data['Label'] 
# train_label = train_label[np.isfinite(train_label).all(1)]
train_data.shape


(625415, 73)

In [41]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_data)
scaler.transform(train_data)

array([[4.46381760e-01, 8.93978843e-03, 7.50202555e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.97009520e-01, 0.00000000e+00, 2.65571704e-02, ...,
        0.00000000e+00, 3.33031335e-02, 0.00000000e+00],
       [4.46433499e-01, 0.00000000e+00, 7.05190401e-04, ...,
        0.00000000e+00, 1.04137378e-05, 0.00000000e+00],
       ...,
       [4.46433499e-01, 0.00000000e+00, 7.70207956e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.46381760e-01, 2.52717441e-02, 1.20032409e-03, ...,
        1.25502008e-03, 1.04137378e-04, 1.00000000e+00],
       [4.46381760e-01, 2.60364510e-02, 9.90267372e-04, ...,
        1.22489960e-03, 4.79031939e-04, 0.00000000e+00]])

In [42]:
print(train_data.shape)
print(data['Label'].shape)

(625415, 73)
(625783,)


In [43]:
from sklearn.model_selection import train_test_split


y = train_data['Label']
X = train_data.drop(columns=['Label'])
# X = train_data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [44]:
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
  
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
  
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [45]:
from sklearn.metrics import accuracy_score

Y_pred = regressor.predict(X_test)
Y_pred = np.where(Y_pred >=0.5, 1, 0)
print(accuracy_score(y_test, Y_pred))

0.999904063855098


In [46]:
import math
 
MSE = np.square(np.subtract(y_test, Y_pred)).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.009794699837256539


In [47]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, f1_score, roc_auc_score

# Mean absolute error (MAE)
mae = mean_absolute_error(y_test, Y_pred)

# Mean squared error (MSE)
mse = mean_squared_error(y_test, Y_pred)

RMSE = math.sqrt(MSE)


# R-squared scores
r2 = r2_score(y_test, Y_pred)

F1 = f1_score(y_test, Y_pred)

roc_auc = roc_auc_score(y_test, Y_pred)



# Print metrics
print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('R-squared scores:', r2)

print("Root Mean Square Error:", RMSE)
print("F1 score:", F1)
print("ROC Auc score:", roc_auc)


Mean Absolute Error: 9.593614490195325e-05
Mean Squared Error: 9.593614490195325e-05
R-squared scores: 0.9983995960391
Root Mean Square Error: 0.009794699837256539
F1 score: 0.9992508615092643
ROC Auc score: 0.9994836155172017


In [48]:
y_pred = np.where(Y_pred >=0.5, 1, 0)
y_pred = pd.DataFrame(y_pred)